In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF

## 1. all

In [3]:
jobs_doc_metadata = pd.read_csv('/home/rep/scRNA-seq_clustering_to_Twitter/P3_Proposed_workflow/PW_files/doc_metadata_stemming_jobs_hclabeled.csv')
del jobs_doc_metadata['Unnamed: 0']

In [4]:
jobs_doc_metadata

,time,text,sn,date,tweets_processed,category,label
0,2009-08-01 10:25:36,Now Hiring: Storage Architect II http://bit.l...,ChicagoJobAds,2009-08-01,hire architect ii,Advertisement,2
1,2009-08-01 22:57:06,"""The Steve Jobs method"" discussion on Hacker N...",hnshah,2009-08-01,steve discuss news via,Junk,1
2,2009-08-01 23:27:08,AZ Jobs | Taco Bell Restaurant General Manager...,ZuluJobsAZ,2009-08-01,az taco bell restaur gener manag taco bell peo...,Advertisement,2
3,2009-08-01 09:55:12,"TN Jobs | SLP Travel Job in Knoxville Area, TN...",ZuluJobsTN,2009-08-01,tn slp travel knoxvil area tn school system so...,Advertisement,2
4,2009-08-01 05:58:39,NJ Jobs | New Jersey Travel or Perm job- OT at...,ZuluJobsNJ,2009-08-01,nj new jersey travel perm ot sunbelt staf nj h...,Advertisement,2
...,...,...,...,...,...,...,...
27739,2009-11-01 02:15:14,these guys have to wake up. make him work alre...,yankee32879,2009-11-01,guy make work alreadi,Personal,5
27740,2009-11-01 03:04:26,Therapy Jobs at HCR! Physical Therapist / PT -...,lydsterj2w,2009-11-01,therapi hcr physic therapist pt prn oh us,Other,4
27741,2009-11-01 00:21:24,hospitality jobs http://bit.ly/3XvUT1,Ur_WebInfoNews,2009-11-01,hospit,News/Politics,3
27742,2009-11-01 03:26:41,Obama Tempers Economic News With Caution On Jo...,suzanne_newton,2009-11-01,obama temper econom news caution money news,News/Politics,3


In [5]:
texts = jobs_doc_metadata['tweets_processed']
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(texts)

DT_tfidf = pd.DataFrame(tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
print(DT_tfidf)

       abc  abl  abt   ac  accept  access  accord  account  ace  across  ...  \
0      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
1      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
2      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
3      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
4      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
...    ...  ...  ...  ...     ...     ...     ...      ...  ...     ...  ...   
27739  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
27740  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
27741  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
27742  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
27743  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   

       yep  yesterday  yet   yo  york  

In [6]:
# DT_tfidf.to_csv("PW_files/doc_word_matrix_stemming_jobs_tfidf.csv")

## 2. main

In [3]:
jobs_doc_metadata = pd.read_csv('PW_files/sub_jobs_main_meta_doc_stemming.csv')
del jobs_doc_metadata['Unnamed: 0']

In [4]:
jobs_doc_metadata

,time,text,sn,date,tweets_processed,category,label,Size_Factor
0,2009-08-01 22:54:37,"I want to see Funny People, but all my friends...",Tabitha_Chase17,2009-08-01,want see funni peopl friend money,Personal,5,0.890852
1,2009-08-01 21:01:56,and not in the 'we don't have jobs b/c we can ...,Myka234,2009-08-01,afford way buy pay rent way,Personal,5,0.890852
2,2009-08-01 20:35:12,"@luckymojo Oh, pleeeease. Hahaha. Held down th...",nataliabadalia,2009-08-01,oh hahaha held fort week need learn cook clean,Personal,5,1.336278
3,2009-08-01 10:16:08,Apple and Steve Jobs to show at CES 2010! Or n...,AppleOnly,2009-08-01,appl steve show,Junk,1,0.445426
4,2009-08-01 16:20:24,So support America. Keep buying American made...,AmericanAuditor,2009-08-01,support america keep buy american made product...,Personal,5,1.781704
...,...,...,...,...,...,...,...,...
11289,2009-11-01 00:31:47,Make Money Online ÛÒ Work From Home Internet ...,Atwatert5g8,2009-11-01,make money onlin uo work home internet day make,Other,4,1.336278
11290,2009-11-01 01:26:27,I concur. RT @edstetzer: CNN: stressful / low ...,AmySullins,2009-11-01,rt cnn stress low pay music top via,Other,4,1.187803
11291,2009-11-01 01:20:45,"And it doesn't help that these ""gang bangers"" ...",TheBelleAgency,2009-11-01,help actual look guy school etc smh,Personal,5,1.039327
11292,2009-11-01 02:35:23,my sql database and cron jobs by cwetony: 1. C...,programingjobs,2009-11-01,sql databas cron copi mysql databas one origin...,Advertisement,2,1.336278


In [5]:
texts = jobs_doc_metadata['tweets_processed']
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(texts)

DT_tfidf = pd.DataFrame(tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
print(DT_tfidf)

       abc  abl  abt   ac  accept  access  accord  account  ace  across  ...  \
0      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
1      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
2      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
3      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
4      0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
...    ...  ...  ...  ...     ...     ...     ...      ...  ...     ...  ...   
11289  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
11290  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
11291  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
11292  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   
11293  0.0  0.0  0.0  0.0     0.0     0.0     0.0      0.0  0.0     0.0  ...   

       yep  yesterday  yet   yo  york  

In [6]:
# DT_tfidf.to_csv("PW_files/doc_word_matrix_stemming_jobs_HDBSCAN_tfidf.csv")

## 3. subtopics

In [7]:
jobssub_doc_metadata = pd.read_csv('PW_files/subsub_jobs_main_meta_doc_stemming.csv')
del jobssub_doc_metadata['Unnamed: 0']

In [8]:
jobssub_doc_metadata

,time,text,sn,date,tweets_processed,category,label,Size_Factor
0,2009-08-01 16:20:24,So support America. Keep buying American made...,AmericanAuditor,2009-08-01,support america keep buy american made product...,Personal,5,1.781704
1,2009-08-01 17:22:11,4 Emerging Green Collar Jobs in Wind Energy ...,maryanneconlin,2009-08-01,emerg green collar wind energi,Other,4,0.742377
2,2009-08-01 16:43:39,Y is THIS pres so intent on destroying free en...,Mike576,2009-08-01,pre destroy free enterpris real solut econom w...,Personal,5,1.484753
3,2009-08-01 06:59:13,"Flex Hour Jobs: Vice President, Membership Dev...",VivianAldana,2009-08-01,flex hour vice presid develop sale inc,Advertisement,2,1.039327
4,2009-08-01 18:25:29,Obama: Recovery won&#39;t come until jobs boun...,Jobresources,2009-08-01,obama recoveri come back intern busi time,Advertisement,2,1.039327
...,...,...,...,...,...,...,...,...
2990,2009-11-01 01:23:39,"Obama says the recession is over, except for t...",LeadSuccess,2009-11-01,obama say recess except part money first half,News/Politics,3,1.187803
2991,2009-11-01 01:49:09,Tell your Senators to support Clean Energy Job...,MIDKNIGHT6,2009-11-01,tell senat support clean energi american power...,Other,4,1.484753
2992,2009-11-01 01:44:53,Paddy Power Bookmaker Information Architecture...,TweetDiscovered,2009-11-01,power inform architectur,Other,4,0.445426
2993,2009-11-01 03:26:08,Obama tempers economic news with caution on jo...,birlaatech,2009-11-01,obama temper econom news caution ap via yahoo ...,News/Politics,3,2.078655


In [10]:
texts = jobssub_doc_metadata['tweets_processed']
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(texts)

DTsub_tfidf = pd.DataFrame(tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
print(DTsub_tfidf)

      abc  abl  abt   ac  access  accord  account  ace  across       act  ...  \
0     0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
1     0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
2     0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
3     0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
4     0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
...   ...  ...  ...  ...     ...     ...      ...  ...     ...       ...  ...   
2990  0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
2991  0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.341585  ...   
2992  0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
2993  0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   
2994  0.0  0.0  0.0  0.0     0.0     0.0      0.0  0.0     0.0  0.000000  ...   

      year  yep  yesterday 

In [11]:
# DTsub_tfidf.to_csv("PW_files/doc_word_matrix_stemming_jobssub_HDBSCAN_tfidf.csv")

## 3. four

In [3]:
four_doc_metadata = pd.read_csv('PW_files/sub_four_main_meta_doc_stemming.csv')
del four_doc_metadata['Unnamed: 0']

In [4]:
four_doc_metadata

,user_id,user_id_new,screen_name,text,tweets_processed,Size_Factor
0,27902825,2,UMichFootball,Leave it all on the field! @UMichFootball! Bes...,leav field umichfootbal best rivalri colleg fo...,1.178521
1,27902825,2,UMichFootball,There’s no time to look backwards… only ahead!...,time look ahead watch umichfootbal ball goblu ...,1.047575
2,27902825,2,UMichFootball,It’s called “The Game’ for a reason. \n\n#GoBl...,call game reason goblu beatosu,0.654734
3,27902825,2,UMichFootball,"On Saturday, our seniors will play their final...",saturday senior play final game big hous senio...,1.309468
4,27902825,2,UMichFootball,The Glasgow Decade with @UMichFootball is just...,glasgow decad umichfootbal complet stori goblu...,0.916628
...,...,...,...,...,...,...
12132,19071682,3,breakingweather,A flash flood emergency is in effect for south...,flash flood emerg effect southwestern arkansa ...,2.226096
12133,19071682,3,breakingweather,"Now that Barry, the first hurricane to make U....",barri first hurrican make landfal year come go...,1.440415
12134,19071682,3,breakingweather,"Showers and locally heavy, drenching thunderst...",shower local heavi drench thunderstorm associ ...,2.095149
12135,19071682,3,breakingweather,While Monday felt like a typical summer day in...,monday felt like typic summer day northeast te...,2.226096


In [5]:
texts = four_doc_metadata['tweets_processed']
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(texts)

DTfour_tfidf = pd.DataFrame(tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
print(DTfour_tfidf)

       abil  abl  abort  absolut  abstin  abus  aca  academ  accept  access  \
0       0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
1       0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
2       0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
3       0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
4       0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
...     ...  ...    ...      ...     ...   ...  ...     ...     ...     ...   
12132   0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
12133   0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
12134   0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
12135   0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   
12136   0.0  0.0    0.0      0.0     0.0   0.0  0.0     0.0     0.0     0.0   

       ...  yesterday  yet  york  young  youth  yum

In [6]:
# DTfour_tfidf.to_csv("PW_files/doc_word_matrix_stemming_four_DBSCAN_tfidf.csv")